In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation,prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Here're just a preparatory phase for prover
- Create dummy data associated with their dataset for verifier to use for generating onnx model.
- Generate data commitment for their dataset.

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [2]
selected_columns = ['x', 'y']
generate_data_commitment(data_path, scales, data_commitment_path)

## Step 2
- User (Verifier) defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `computation_to_model()` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`, while also export `precal_witness_arr.json` which is the constant witness needed for verifier to generate the same onnx.

In [6]:
from zkstats.computation import State, computation_to_model


def computation(state: State, x: list[torch.Tensor]):
    out_0 = torch.sum(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.cat((out_0.unsqueeze(0), out_1.unsqueeze(0))).reshape(1,-1,1))

error = 0.01

_, prover_model = computation_to_model(computation,precal_witness_path, True, error )
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:249: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inf

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 2, param_scale: 2, scale_input_multiplier: 10) ------------->

+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.012499809 | 0.024999619  | 0.024999619 | 0         | 0.012499809    | 0.024999619      | 0.024999619   | 0             | 0.00031249048      | 0.0009979887       | 0.0009979887           |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---

scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-56,64],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":2624,"total_assignments":812,"total_const_size":168,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2,2],"module_sizes":{"kzg":[],"poseidon":[2624,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"ReLU","Abs",{"Floor":{"scale":8.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715672095084}


## Step 3
- User (Verifier) gets `precal_witness_arr.json`, and generate the same onnx format with same parameter.

In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False,error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

## Step 4
- Verifier sets up vk, pk.
- Prover generates proof with `prover_gen_proof`


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path, vk_path, pk_path )

print("=======================================")
# Prover generates proof
# print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.6220359802246094 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 12.5
==== Generating Proof ====
proof:  {'instances': [['cde936180fb7e379a578309232773e02b017d59f9001712b917a148b525d7b19', 'a38c8628cd223f38f854eade2722b8dd09b5797a0408398dd3d5160b6584e90b', '0100000000000000000000000000000000000000000000000000000000000000', '3200000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x04e788bb02858310a749cffae4a10707df97bd2f4cbcd25d1ae77a91745cf1b625124a4099ee70b7ea08704d16f98955474883b249ae30e471e8d29b4c00c6b92c31afb2f95fc73f2464030d710a977561c3c9e87d82e08d5170c88e6fbb8b991a436f823e1b793a32e3845fd3c02f547f3bd6de661547bd2e1bae4d9a78bb9420ac32cb316fd3282ec17b06d6daed30991a679e955302e0226bbf01bd4d0ec52c3851ba90f370f0b4113f63ce41999cbd261edf44743fdcf9b52847763bcc7a11815eda358d8caa851714802e7c6a676091d573ec5efa73ec91e25b4b7c272601dac029a29452998ca66a574e979cf59419f8afc587f46e185b6d5c8f13479d06faeb2d129

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)

[12.5]